## A simple swarm

In [14]:
%load_ext autoreload 
%autoreload 2
import sys
sys.tracebacklimit = 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from core.agent import *
from core.world import * 
from core.map import *
import numpy as np
from core.render import * 

In [16]:
def initialize_swarm(world : World):
    swarm = [Agent() for i in range(1000)]
    for agent in swarm:
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)
    

In [17]:
world = World(dims = (100, 100),
              swarm_initialzier= initialize_swarm,
              resource_generator= RandomMapGenerator(),
              energy_model=EnergyModel()
              )
world.reset()

In [18]:
# Test movements by doing random actions
def update():
    for agent in world.agents:
        action_choice = np.random.randint(0, 3)
        choice = np.random.randint(1, 5)

        match (action_choice): 
            case 1: agent.move(choice)
            case 2: agent.pick_up(choice)
            case 3: agent.put_down(choice)

    world.update()

update()

In [19]:
render_world(world, (800, 800), update_fn=update, delay_s=0)

## Verify certain things about the simulation

In [20]:
swarm = world.agents

In [21]:
# Validate the observation space provided is sensible
obs = swarm[0].local_observation

print(obs.nearby_agents, obs.nearby_agents.shape)
print(obs.resource_types, obs.resource_types.shape)

[[  0   0   0   0   0   0 108]
 [  0   0   0 594   0   0   0]
 [  0 516   0   0   0   0   0]
 [  0   0 949   0   0 349   0]
 [  0   0 426   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0 314   0   0]] (7, 7)
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]] (7, 7)


In [22]:
# Validate that all swarm agents landed in different positions
positions = set()
for agent in swarm:
    pos = agent.current_position
    positions.add((pos[0], pos[1]))

print(len(positions), len(world.agents))

1000 1000


In [23]:
# Check for resources
rsrc_qty = world._resource_map._resource_quantity_map
qtys = []

for i in range(rsrc_qty.shape[0]):
    for j in range(rsrc_qty.shape[1]):
        if rsrc_qty[i, j] > 0:
            qtys.append(rsrc_qty[i, j])

print(qtys)

[6.342046196481015, 6.118137467170772, 1.0, 6.4325916690094775, 2.829878981337668, 3.045434225994317, 4.338444432674269, 4.337158920440771, 4.289590581303414, 5.271676583987374, 5.123910294470734, 4.063943825514564, 1.13760317477825, 2.733475031286914, 8.45140159843697, 6.2789464309796825, 4.992011898811841, 6.047531018843178, 4.209465569261455, 7.411145408990189, 3.744329094654933, 7.931681819533592, 1.7580744432791073, 4.900851322125036, 4.6225064679368595, 6.2956654815769895, 3.6089318658375618, 3.8913178150588665, 1.7405241884718121, 7.39328157651126, 5.58299396153956, 2.7247265997541725, 5.949949838071809, 3.9693970695010243, 3.9317749203806436, 3.271935324992402, 5.4332839358475225, 4.436689724973061, 5.348921868833422, 3.757217818705191, 3.887514007092075, 3.579694759007275, 4.064120722866397, 2.129368591235773, 4.052804095322545, 2.791582484993466, 3.2943020676541965, 10.243487457374407, 9.232978108063726, 5.074645375069774, 8.41330914566933, 3.7841242604707386, 3.2120583752799

In [24]:
# Check for agent state
state = swarm[0]._current_state.inventory
print(state)


[]


# Perftest

In [26]:
import cProfile
def stress_test():

    for _ in range(0, 1):
        update()

In [27]:
world.reset()
cProfile.run('stress_test()', sort = 'time')

         52236212 function calls in 23.854 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  2000000    2.869    0.000    2.869    0.000 {method 'randint' of 'numpy.random.mtrand.RandomState' objects}
     1000    2.064    0.002    4.915    0.005 world.py:85(_update_movement)
  1000000    1.764    0.000    3.117    0.000 world.py:126(_get_nearby_agents)
  1000000    1.726    0.000    3.030    0.000 world.py:146(_get_nearby_resources)
  6385293    1.592    0.000    1.592    0.000 {method 'copy' of 'numpy.ndarray' objects}
     1000    1.381    0.001   19.261    0.019 world.py:48(update)
  6385293    1.176    0.000    2.768    0.000 agent.py:205(current_position)
  2666745    0.947    0.000    0.947    0.000 {built-in method numpy.array}
  3998695    0.820    0.000    1.077    0.000 enum.py:193(__get__)
   329645    0.810    0.000    0.916    0.000 map.py:43(subtract_resource)
  5390268    0.717    0.000    0.717    0.000 {bu